# Analysing FES model tidal current velocity 

In [76]:
import xarray as xr 
import matplotlib.pyplot as plt 
import numpy as np 
import os
import matplotlib
matplotlib.use('Agg')

## 1. - Analysis of M2 and K1  

In [78]:
m2_u = xr.open_dataset("/bettik/bellemva/FES_tide/eastward_velocity/m2.nc",drop_variables=["lat_bnds","lon_bnds","crs"])
m2_v = xr.open_dataset("/bettik/bellemva/FES_tide/northward_velocity/m2.nc",drop_variables=["lat_bnds","lon_bnds","crs"])

k1_u = xr.open_dataset("/bettik/bellemva/FES_tide/eastward_velocity/k1.nc",drop_variables=["lat_bnds","lon_bnds","crs"])
k1_v = xr.open_dataset("/bettik/bellemva/FES_tide/northward_velocity/k1.nc",drop_variables=["lat_bnds","lon_bnds","crs"])

Selection around Hawai'i :

In [79]:
m2_u = m2_u.sel(lon = slice(185,205),lat = slice(15,35), drop=True)
m2_v = m2_v.sel(lon = slice(185,205),lat = slice(15,35), drop=True)

k1_u = k1_u.sel(lon = slice(185,205),lat = slice(15,35), drop=True)
k1_v = k1_v.sel(lon = slice(185,205),lat = slice(15,35), drop=True)

File of internal tide (for lon lat grid) 

In [80]:
ds = xr.open_dataset("/bettik/bellemva/MITgcm/MITgcm_it/by_mode/MITgcm_it_20120601.nc")
longitude = ds.longitude.values
latitude = ds.latitude.values

In [81]:
m2_u = m2_u.interp({"lon":longitude,"lat":latitude})
m2_v = m2_v.interp({"lon":longitude,"lat":latitude})

k1_u = k1_u.interp({"lon":longitude,"lat":latitude})
k1_v = k1_v.interp({"lon":longitude,"lat":latitude})

In [82]:
print(m2_u.Ug.values[121,121])
print(m2_v.Vg.values[121,121])

-85.09625
-143.42293


In [83]:
w_m2 = 2*np.pi/(12.42*3600)
w_k1 = 2*np.pi/(23.93*3600)

In [84]:
#nb_period = 4
#array_t=np.arange(0,18*29.93*3600+1,3600)
array_t=np.arange(0,12.42*3600+1,3600/6)


Bathymetry dataset 

In [85]:
bathy = xr.open_dataset("/bettik/bellemva/sad/Bathymetry_hawai.nc").elevation
bathy["lon"] = bathy["lon"]+360
bathy = bathy.interp({"lon" : longitude, "lat" : latitude},method='nearest',kwargs={"fill_value": "extrapolate"})
bathy = bathy.load()
bathy = bathy.rename({'lon': 'longitude','lat': 'latitude'})

In [86]:
y_grad, x_grad = np.gradient(bathy.values)

In [87]:
norm_grad = np.sqrt(x_grad**2+y_grad**2)

In [88]:
np.argsort(norm_grad.flatten())

array([440360, 194577, 624684, ..., 517735,   3932, 127899])

Internal tide generation for biggest graident sites : 

In [89]:
#index of bigger gradients
idx1 = np.unravel_index(norm_grad.flatten().argsort()[-1], norm_grad.shape) # (first biggest gradient)
idx2 = np.unravel_index(norm_grad.flatten().argsort()[-2], norm_grad.shape) # (second biggest gradient)
idx3 = np.unravel_index(norm_grad.flatten().argsort()[-3], norm_grad.shape) # (third biggest gradient)

In [ ]:
array_itg1 = []
array_itg2 = []
array_itg3 = []

for t in array_t :
    for idx,array_itg in zip([idx1,idx2,idx3],[array_itg1,array_itg2,array_itg3]):
        #array_itg.append(  (m2_u.Ua.values[idx]*np.cos(w_m2*t+m2_u.Ug.values[idx])+k1_u.Ua.values[idx]*np.cos(w_k1*t+k1_u.Ug.values[idx]))*x_grad[idx] + \
        #                   (m2_v.Va.values[idx]*np.cos(w_m2*t+m2_v.Vg.values[idx])+k1_v.Va.values[idx]*np.cos(w_k1*t+k1_v.Vg.values[idx]))*y_grad[idx]  )
        array_itg.append(  m2_u.Ua.values[idx]*np.cos(w_m2*t+m2_u.Ug.values[idx])*x_grad[idx] + \
                           m2_v.Va.values[idx]*np.cos(w_m2*t+m2_v.Vg.values[idx])*y_grad[idx]  )


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(array_t,array_itg1,label="Point 1")
ax.plot(array_t,array_itg2,label="Point 2")
ax.plot(array_t,array_itg3,label="Point 3")

ax.set_xlabel("Time[second]")
ax.set_ylabel("Internal Tide generation")

ax.legend()


Internal tide generation over all Hawai'i

In [90]:
results = np.zeros((array_t.size,ds.dims['latitude'],ds.dims['longitude']))

In [91]:
for i,t in enumerate(array_t) : 
    #results[i] = (m2_u.Ua.values*np.cos(w_m2*t+m2_u.Ug.values)+k1_u.Ua.values*np.cos(w_k1*t+k1_u.Ug.values))*x_grad + \
    #                       (m2_v.Va.values*np.cos(w_m2*t+m2_v.Vg.values)+k1_v.Va.values*np.cos(w_k1*t+k1_v.Vg.values))*y_grad
    results[i] = m2_u.Ua.values*np.cos(w_m2*t+m2_u.Ug.values)*x_grad + \
                           m2_v.Va.values*np.cos(w_m2*t+m2_v.Vg.values)*y_grad


In [98]:
def form(i):
    if i//10==0:
        return "0000"+str(i)
    elif i//100==0 : 
        return "000"+str(i)
    elif i//1000==0 : 
        return "00"+str(i)
    elif i//10000==0 : 
        return "0"+str(i)
    else : 
        return str(i)

In [99]:
def plot(t,array):

    fig,ax = plt.subplots(1,1,figsize=(6,6))
    plot = ax.pcolormesh(longitude,latitude,array,vmin=-600,vmax=600,cmap='RdBu')
    plt.colorbar(plot,shrink=0.8)

    ax.set_aspect("equal")
    ax.set_xlabel("Longitude [°]",fontsize=12)
    ax.set_ylabel("Latitude [°]",fontsize=12)
    ax.set_title("Internal Tide Generation")

    plt.savefig("./frames/"+form(int(t))+".png")

In [100]:
for i,t in enumerate(array_t):
    plot(t,results[i,:,:])

In [ ]:
plt.hist(results[0,:,:].flatten(),range = (-200,200),bins = 100)

## 2. - Principal component analysis 

In [ ]:
name_file = list(set(os.listdir('/bettik/bellemva/FES_tide/northward_velocity/')) \
                 .intersection(os.listdir('/bettik/bellemva/FES_tide/eastward_velocity/')))

In [ ]:
longitude = np.arange(185.,205.+1/50,1/48)
latitude = np.arange(15.,35.+1/50,1/48)

def magnitude_tide (name_file) : 
    ds_u = xr.open_dataset("/bettik/bellemva/FES_tide/eastward_velocity/"+name_file,drop_variables=["lat_bnds","lon_bnds","crs"])
    ds_v = xr.open_dataset("/bettik/bellemva/FES_tide/northward_velocity/"+name_file,drop_variables=["lat_bnds","lon_bnds","crs"])

    ds_u = ds_u.sel(lon = slice(185,205),lat = slice(15,35), drop=True)
    ds_v = ds_v.sel(lon = slice(185,205),lat = slice(15,35), drop=True)

    ds_u = ds_u.interp({"lon":longitude,"lat":latitude})
    ds_v = ds_v.interp({"lon":longitude,"lat":latitude})
    
    return np.nanmean(np.sqrt(ds_u.Ua.values**2+ds_v.Va.values**2))

In [ ]:

result_magnitude = [magnitude_tide(name) for name in name_file]

In [ ]:
import pandas as pd 
ds_tides = pd.DataFrame(data={'name': name_file,'magnitude' : result_magnitude})
ds_tides = ds_tides.sort_values("magnitude",ascending = False)
ds_tides["cumsum"] = ds_tides["magnitude"].cumsum()
ds_tides["cumper"] = 100*ds_tides["cumsum"]/ds_tides["magnitude"].sum()

In [ ]:
ds_tides